# Hierarchical double Poisson → Neyman Type A

We simulate and analyze the **marginal distribution of predicted counts** under:

1. **True number of rays** \(N \) ~ Poisson(λ)
2. **Predicted count** \(Y \mid N \) ~ Poisson(N)

So: draw a true count \(N\) from Poisson(λ), then draw the prediction \(Y\) from Poisson with mean \(N\). This is a hierarchical (compound) double Poisson.

**Result:** The marginal distribution of \(Y\) is the **Neyman Type A** distribution with parameters \((\lambda, \phi=1)\). In the general Neyman Type A, you have Poisson(λ) clusters and each cluster contributes Poisson(φ) individuals; here the "cluster" is the true count and we draw a single Poisson(N), which corresponds to \(\phi=1\) (one Poisson draw per "cluster").

**Properties:**
- \(\mathbb{E}[Y] = \lambda\)
- \(\mathrm{Var}(Y) = \lambda + \lambda = 2\lambda\) (overdispersed relative to Poisson)
- Index of dispersion: \(\mathrm{Var}(Y)/\mathbb{E}[Y] = 2\)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import special

## 1. Simulate the hierarchical double Poisson

For each sample: draw \(N \sim \mathrm{Poisson}(\lambda)\), then \(Y \sim \mathrm{Poisson}(N)\).

In [ ]:
def simulate_hierarchical_double_poisson(lambda_: float, size: int, rng: np.random.Generator):
    """
    True count N ~ Poisson(lambda), predicted Y | N ~ Poisson(N).
    Returns (true_counts, predicted_counts).
    """
    true_counts = rng.poisson(lambda_, size=size)
    predicted_counts = rng.poisson(true_counts)
    return true_counts, predicted_counts


lambda_true = 3.0
n_samples = 50_000
rng = np.random.default_rng(42)
true_counts, predicted_counts = simulate_hierarchical_double_poisson(lambda_true, n_samples, rng)

In [ ]:
print("Simulated predicted counts:")
print(f"  Mean:    {predicted_counts.mean():.4f}  (theory: {lambda_true})")
print(f"  Var:     {predicted_counts.var():.4f}  (theory: {2 * lambda_true})")
print(f"  Dispersion (Var/Mean): {predicted_counts.var() / (predicted_counts.mean() or 1):.4f}  (theory: 2)")

## 2. Theoretical marginal PMF: Neyman Type A with φ = 1

The Neyman Type A PMF is

\begin{equation}
P(Y = x) = \frac{e^{-\lambda} \phi^x}{x!} \sum_{j=0}^{\infty} \frac{(\lambda e^{-\phi})^j \, j^x}{j!}
\end{equation}

With \(\phi = 1\) this becomes

\begin{equation}
P(Y = x) = \frac{e^{-\lambda}}{x!} \sum_{j=0}^{\infty} \frac{(\lambda e^{-1})^j \, j^x}{j!}
\end{equation}

We implement this by truncating the sum over \(j\) at a sufficiently large value.

In [ ]:
def neyman_type_a_pmf(x: np.ndarray, lam: float, phi: float = 1.0, j_max: int = 200) -> np.ndarray:
    """
    Neyman Type A PMF: P(Y=x) for x in the array.
    With phi=1 this is the marginal of the hierarchical double Poisson.
    """
    out = np.zeros_like(x, dtype=float)
    j_grid = np.arange(0, j_max + 1, dtype=float)
    coeff = (lam * np.exp(-phi)) ** j_grid / special.factorial(j_grid)
    for i, xi in enumerate(np.ravel(x)):
        xi = int(xi)
        if xi < 0:
            out.flat[i] = 0.0
            continue
        s = np.sum(coeff * (j_grid ** xi))
        out.flat[i] = (np.exp(-lam) * (phi ** xi) / special.factorial(xi)) * s
    return out

In [ ]:
# PMF over a range of counts
x_max = int(min(predicted_counts.max() + 5, 30))
x_vals = np.arange(0, x_max + 1)
pmf_theory = neyman_type_a_pmf(x_vals, lambda_true, phi=1.0)
print("Theoretical Neyman Type A(λ, φ=1) PMF (first 12 values):")
for x, p in zip(x_vals[:12], pmf_theory[:12]):
    print(f"  P(Y={x}) = {p:.6f}")
print(f"  Sum (0..{x_max}) = {pmf_theory.sum():.6f}")

## 3. Compare simulated histogram to theoretical PMF

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Left: histogram of predicted counts vs theoretical PMF
ax = axes[0]
bins = np.arange(-0.5, x_max + 1.5, 1.0)
ax.hist(predicted_counts, bins=bins, density=True, alpha=0.7, color="steelblue", edgecolor="white", label="Simulated")
ax.scatter(x_vals, pmf_theory, color="coral", s=60, zorder=5, label="Neyman Type A(λ, φ=1)")
ax.set_xlabel("Predicted count Y")
ax.set_ylabel("Probability / density")
ax.set_title(f"Marginal of Y: hierarchical double Poisson (λ={lambda_true})")
ax.legend()
ax.set_xlim(-0.5, x_max + 0.5)

# Right: (True N, Predicted Y) joint view
ax = axes[1]
ax.hexbin(true_counts, predicted_counts, gridsize=25, mincnt=1, cmap="Blues")
ax.set_xlabel("True count N")
ax.set_ylabel("Predicted count Y")
ax.set_title("Joint (N, Y) from simulation")
ax.set_aspect("equal")

plt.tight_layout()
plt.show()

## 4. Summary

- **Model:** True rays \(N \sim \mathrm{Poisson}(\lambda)\), predicted \(Y \mid N \sim \mathrm{Poisson}(N)\).  
- **Marginal of \(Y\):** Neyman Type A with parameters \((\lambda, \phi=1)\).  
- **Mean = λ**, **Variance = 2λ**, so predictions are overdispersed (variance twice the mean).  
- This is the same as the compound Poisson where the number of "clusters" is Poisson(λ) and each cluster contributes a single Poisson(1) draw—i.e. one Poisson(N) draw when the cluster size is N.